# Money Modeling Software v1.0 - Complete Example with Firm Agents

This notebook implements the concrete example from pages 7-10 of the Money modeling software.pdf document
using the new Firm agent type for proper business entity representation.

## Example Timeline

- **t0**: Initial setup with 4 firm agents
- **t1**: Agent 3 delivers 10 machines to Agent 2
- **t2**: Agent 4 pays $110 to Agent 2; Agent 2 delivers 10 machines to Agent 4
- **t3**: Agent 2 pays $110 to Agent 1 (bond settlement)

## Expected Final State
- Agent 1: $110 cash
- Agent 2: Nothing
- Agent 3: Nothing
- Agent 4: 10 machines

In [ ]:
# Import required modules
from bilancio.engines.system import System
from bilancio.engines.simulation import run_day
from bilancio.domain.agents.firm import Firm  # Using Firm for business entities
from bilancio.domain.agents.central_bank import CentralBank
from bilancio.domain.instruments.credit import Payable
from bilancio.domain.instruments.nonfinancial import Deliverable
from decimal import Decimal

# Initialize the system
sys = System()
print(f"System initialized at day {sys.state.day}")

In [ ]:
# Create Central Bank (required for cash issuance)
cb = CentralBank(id="CB", name="Central Bank", kind="central_bank")
sys.add_agent(cb)

# Create the 4 agents as Firms (business entities)
agent1 = Firm(id="Agent1", name="Firm 1", kind="firm")
agent2 = Firm(id="Agent2", name="Firm 2", kind="firm") 
agent3 = Firm(id="Agent3", name="Manufacturing Firm 3", kind="firm")  # Has machines
agent4 = Firm(id="Agent4", name="Trading Firm 4", kind="firm")  # Has cash

sys.add_agent(agent1)
sys.add_agent(agent2)
sys.add_agent(agent3)
sys.add_agent(agent4)

print("✓ All agents created as Firms")
for agent_id in ["Agent1", "Agent2", "Agent3", "Agent4"]:
    print(f"  - {agent_id}: {sys.state.agents[agent_id].name} (kind={sys.state.agents[agent_id].kind})")

## Set up Initial Balance Sheet Positions (t0)

In [ ]:
# 1. Bond: Agent 2 owes Agent 1 $110 (due day 3 = t3)
bond = Payable(
    id=sys.new_contract_id("BOND"),
    kind="payable",
    amount=110,
    denom="USD",
    asset_holder_id="Agent1",
    liability_issuer_id="Agent2",
    due_day=3
)
sys.add_contract(bond)
print("✓ Bond created: Agent2 owes Agent1 $110 (due day 3)")

# 2. Agent 3's machine inventory (manufacturing firm with production assets)
machines_inv = Deliverable(
    id=sys.new_contract_id("MACH"),
    kind="deliverable",
    amount=10,
    sku="machines",
    unit_price=Decimal("0"),
    denom="USD",
    asset_holder_id="Agent3",
    liability_issuer_id="Agent3",  # Self-issued inventory
    due_day=None
)
sys.add_contract(machines_inv)
print("✓ Inventory: Agent3 (manufacturer) owns 10 machines")

# 3. Deliverable obligation: Agent 3 owes Agent 2 10 machines (due day 1 = t1)
delivery1 = Deliverable(
    id=sys.new_contract_id("DEL1"),
    kind="deliverable",
    amount=10,
    sku="machines",
    unit_price=Decimal("0"),
    denom="USD",
    asset_holder_id="Agent2",
    liability_issuer_id="Agent3",
    due_day=1
)
sys.add_contract(delivery1)
print("✓ Delivery contract: Agent3 must deliver 10 machines to Agent2 (due day 1)")

# 4. Agent 4's cash (trading firm with liquidity)
sys.mint_cash("Agent4", 110)
print("✓ Cash minted: Agent4 (trader) has $110")

# 5. Payable: Agent 4 owes Agent 2 $110 (due day 2 = t2)
payment = Payable(
    id=sys.new_contract_id("PAY"),
    kind="payable",
    amount=110,
    denom="USD",
    asset_holder_id="Agent2",
    liability_issuer_id="Agent4",
    due_day=2
)
sys.add_contract(payment)
print("✓ Payment obligation: Agent4 owes Agent2 $110 (due day 2)")

# 6. Deliverable obligation: Agent 2 owes Agent 4 10 machines (due day 2 = t2)
delivery2 = Deliverable(
    id=sys.new_contract_id("DEL2"),
    kind="deliverable",
    amount=10,
    sku="machines",
    unit_price=Decimal("0"),
    denom="USD",
    asset_holder_id="Agent4",
    liability_issuer_id="Agent2",
    due_day=2
)
sys.add_contract(delivery2)
print("✓ Delivery contract: Agent2 must deliver 10 machines to Agent4 (due day 2)")

print("\n📊 All initial positions established")

## Display Initial State (t0)

In [ ]:
def show_balance_sheet(sys, agent_id, show_details=True):
    """Display balance sheet for an agent."""
    agent = sys.state.agents[agent_id]
    
    assets = []
    liabilities = []
    
    for cid in agent.asset_ids:
        c = sys.state.contracts.get(cid)
        if not c:
            continue
        if c.kind == "cash":
            assets.append(f"Cash: ${c.amount}")
        elif c.kind == "payable":
            assets.append(f"Receivable from {c.liability_issuer_id}: ${c.amount} (due day {c.due_day})")
        elif c.kind == "deliverable":
            if c.liability_issuer_id == agent_id:
                assets.append(f"Inventory: {c.amount} {c.sku}")
            else:
                assets.append(f"Claim on {c.liability_issuer_id}: {c.amount} {c.sku} (due day {c.due_day})")
    
    for cid in agent.liability_ids:
        c = sys.state.contracts.get(cid)
        if not c or c.asset_holder_id == agent_id:  # Skip self-issued
            continue
        if c.kind == "payable":
            liabilities.append(f"Payable to {c.asset_holder_id}: ${c.amount} (due day {c.due_day})")
        elif c.kind == "deliverable":
            liabilities.append(f"Deliver to {c.asset_holder_id}: {c.amount} {c.sku} (due day {c.due_day})")
    
    if show_details:
        print(f"\n{agent_id} ({sys.state.agents[agent_id].name}):")
        print("  Assets:")
        for a in assets:
            print(f"    • {a}")
        if not assets:
            print("    • None")
        print("  Liabilities:")
        for l in liabilities:
            print(f"    • {l}")
        if not liabilities:
            print("    • None")
    
    return assets, liabilities

print("="*70)
print("INITIAL BALANCE SHEETS (t0 = Day 0)")
print("="*70)

for agent_id in ["Agent1", "Agent2", "Agent3", "Agent4"]:
    show_balance_sheet(sys, agent_id)

## Run Simulation Day by Day

In [ ]:
# Day 1 (t1): Machine delivery
print("\n" + "="*70)
print("DAY 1 (t1): Manufacturing delivery")
print("Expected: Agent3 delivers 10 machines to Agent2")
print("="*70)

# Run day - should automatically settle deliverable due on day 1
run_day(sys)

# Check what happened
events_day1 = [e for e in sys.state.events if e.get("kind") == "DeliverableSettled"]
for event in events_day1:
    print(f"\n✓ Settlement executed: {event['debtor']} → {event['creditor']}")
    print(f"  Delivered: {event.get('quantity', 'N/A')} {event.get('sku', 'items')}")

print(f"\nSystem advanced to day {sys.state.day}")
print("\nAgent2 after receiving machines:")
assets, _ = show_balance_sheet(sys, "Agent2", show_details=False)
for a in assets:
    print(f"  • {a}")

In [ ]:
# Day 2 (t2): Payment and machine delivery
print("\n" + "="*70)
print("DAY 2 (t2): Payment and delivery exchange")
print("Expected: Agent4 pays $110 to Agent2")
print("Expected: Agent2 delivers 10 machines to Agent4")
print("="*70)

# Run day - should settle both payment and deliverable
run_day(sys)

# Check settlements
recent_events = sys.state.events[-10:]
for event in recent_events:
    if event.get("kind") == "PayableSettled":
        print(f"\n✓ Payment settled: {event['debtor']} → {event['creditor']}: ${event['amount']}")
    elif event.get("kind") == "DeliverableSettled" and event not in events_day1:
        print(f"\n✓ Delivery settled: {event['debtor']} → {event['creditor']}")
        print(f"  Delivered: {event.get('quantity', 'N/A')} {event.get('sku', 'items')}")

print(f"\nSystem advanced to day {sys.state.day}")

# Show Agent2's cash and Agent4's machines
print("\nAgent2 after receiving payment:")
assets2, _ = show_balance_sheet(sys, "Agent2", show_details=False)
for a in assets2:
    print(f"  • {a}")

print("\nAgent4 after receiving machines:")
assets4, _ = show_balance_sheet(sys, "Agent4", show_details=False)
for a in assets4:
    print(f"  • {a}")

In [ ]:
# Day 3 (t3): Bond settlement
print("\n" + "="*70)
print("DAY 3 (t3): Bond maturity")
print("Expected: Agent2 pays $110 to Agent1 (bond settlement)")
print("="*70)

# Run day - should settle the bond
run_day(sys)

# Check bond settlement
for event in sys.state.events[-5:]:
    if event.get("kind") == "PayableSettled" and event.get("amount") == 110:
        if event not in recent_events:  # Make sure it's the new one
            print(f"\n✓ Bond settled: {event['debtor']} → {event['creditor']}: ${event['amount']}")

print(f"\nSystem advanced to day {sys.state.day}")

## Final State Verification

In [ ]:
print("\n" + "="*70)
print("FINAL BALANCE SHEETS (After t3)")
print("="*70)

final_states = {}
for agent_id in ["Agent1", "Agent2", "Agent3", "Agent4"]:
    assets, liabilities = show_balance_sheet(sys, agent_id)
    final_states[agent_id] = {"assets": assets, "liabilities": liabilities}

print("\n" + "="*70)
print("VERIFICATION AGAINST EXPECTED")
print("="*70)

# Verify each agent
checks = []

# Agent 1 should have $110 cash
a1_cash = sum(c.amount for cid in sys.state.agents["Agent1"].asset_ids 
              if (c := sys.state.contracts.get(cid)) and c.kind == "cash")
check1 = a1_cash == 110
checks.append(check1)
print(f"\nAgent1 has $110 cash: {'✅' if check1 else '❌'} (actual: ${a1_cash})")

# Agent 2 should have nothing
a2_assets = [c for cid in sys.state.agents["Agent2"].asset_ids if sys.state.contracts.get(cid)]
a2_liabs = [c for cid in sys.state.agents["Agent2"].liability_ids 
            if (c := sys.state.contracts.get(cid)) and c.asset_holder_id != "Agent2"]
check2 = len(a2_assets) == 0 and len(a2_liabs) == 0
checks.append(check2)
print(f"Agent2 has no assets/liabilities: {'✅' if check2 else '❌'}")

# Agent 3 should have nothing
a3_assets = [c for cid in sys.state.agents["Agent3"].asset_ids if sys.state.contracts.get(cid)]
a3_liabs = [c for cid in sys.state.agents["Agent3"].liability_ids 
            if (c := sys.state.contracts.get(cid)) and c.asset_holder_id != "Agent3"]
check3 = len(a3_assets) == 0 and len(a3_liabs) == 0
checks.append(check3)
print(f"Agent3 has no assets/liabilities: {'✅' if check3 else '❌'}")

# Agent 4 should have 10 machines
a4_machines = sum(c.amount for cid in sys.state.agents["Agent4"].asset_ids 
                  if (c := sys.state.contracts.get(cid)) and c.kind == "deliverable")
check4 = a4_machines == 10
checks.append(check4)
print(f"Agent4 has 10 machines: {'✅' if check4 else '❌'} (actual: {a4_machines})")

print("\n" + "="*70)
if all(checks):
    print("🎉 SUCCESS! The simulation perfectly matches the PDF example!")
    print("\nKey achievements:")
    print("✓ Firm agents can hold cash and settle payments")
    print("✓ Deliverables automatically settle on due dates")
    print("✓ All temporal obligations executed correctly")
    print("✓ Final state exactly matches specification")
    print("\n📊 The bilancio system successfully implements Version 1.0 requirements!")
else:
    print("⚠️  Some checks failed. Debugging needed.")
    print("\nCheck the event log for details:")
    
    # Show all settlement events
    print("\nAll settlements:")
    for i, event in enumerate(sys.state.events):
        if "Settled" in event.get("kind", ""):
            print(f"  {i}. {event}")